# **Stop Words & Tokenizing(2019)**

In [1]:
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# DB 연결
# !pip install pymysql
import pymysql

# DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

# 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

# 텍스트 분석
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re

# 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

# 모델 저장 및 로드
import joblib

# 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
maeil = pd.read_csv('../../../../Code/Data/2019_maeil.csv')
asia = pd.read_csv('../../../../Code/Data/2019_asia.csv')

In [3]:
maeil[maeil['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,score,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed
4949,현대차,5380,매일경제,2019021914,"이시한, 취업계 2년 연속 트렌드 예측 경향성 소름 돋는 적중, ‘문어시한’이 떴다...",http://edu.mk.co.kr/edunews/news_view.php?sc=5...,NaN,2019-02-19,14,0,119500,120500,119000,119500,470357,-0.008299,-1,0
4956,현대차,5380,매일경제,2019022011,"이시한이 말하는, 100:1이 일반화된 채용시장에서 살아남는 방법 - 인터뷰 #2",http://edu.mk.co.kr/edunews/news_view.php?sc=5...,NaN,2019-02-20,11,0,119500,121500,119000,119000,735087,-0.004184,-1,0
9058,sk하이닉스,660,매일경제,2019112017,이시한의 2020년 취업트렌드코리아 #2 : Implement AI,http://edu.mk.co.kr/edunews/news_view.php?sc=5...,NaN,2019-11-21,17,0,81700,82500,80500,80900,3346746,-0.021765,-1,0


In [4]:
asia[asia['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,score,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed


In [5]:
maeil.dropna(axis=0, inplace=True)   
maeil = maeil.reset_index(drop=True)   # reset_index() 꼭 해줘야 함...!!
asia.dropna(axis=0, inplace=True)
asia = asia.reset_index(drop=True)     # reset_index() 꼭 해줘야 함...!!

In [6]:
media_DF_list = [maeil, asia]

In [7]:
# Stop Words List에 각 매체명 추가
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt")
media_list = ['매일경제', '아시아경제', '삼프로TV', '슈카월드', '한국경제TV']
for word in [media_list]:
    stopwords.append(word)

In [8]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [9]:
for i in range(len(media_DF_list)):
    media_DF_list[i]['except_stopwords'] = 0
    for j in range(len(media_DF_list[i]['text'])):
#         hangeul = re.compile('[^ ㄱ-ㅣ 가-힣]')                  # 정규 표현식  → 한글 추출 규칙 : 띄어쓰기(1개)를 포함한 한글
#         result = hangeul.sub('', media_DF_list[i]['text'][j])   # 위에 설정한 hangeul 규칙을 text에 적용
        okt = Okt()                                             # 형태소 추출
        nouns = okt.nouns(media_DF_list[i]['text'][j])
        nouns = [x for x in nouns if len(x) > 1]                # 한 글자 키워드 제거
        nouns = [x for x in nouns if x not in stopwords]        # 불용어 제거
        
        corpus = " ".join(nouns)                                # List를 String으로 변환
        media_DF_list[i]['except_stopwords'][j] = corpus

In [10]:
maeil.to_csv('../../../../Code/Data/Test/2019_maeil.csv', index=False)
asia.to_csv('../../../../Code/Data/Test/2019_asia.csv', index=False)